In [1]:
!pip install --upgrade txtai

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import numpy as np
from datetime import datetime, timedelta
import heapq
import random
from txtai.embeddings import Embeddings
import random

In [3]:
class Patient:
    def __init__(self, id, location, health_status, video_description, text_description, required_doctor_type, score, is_emergency=False):
        self.id = id
        self.location = location
        self.health_status = health_status
        self.video_description = video_description
        self.text_description = text_description
        self.required_doctor_type = required_doctor_type
        self.score = float(score)
        self.is_emergency = is_emergency
        self.application_date = datetime.now()
        self.appointment = None
        self.last_reschedule_date = None

    @property
    def waiting_time(self):
        if self.appointment:
            wait_time = max((self.appointment.datetime - self.application_date).days, 0)
        elif self.last_reschedule_date:
            wait_time = max((datetime.now() - self.last_reschedule_date).days, 0)
        else:
            wait_time = max((datetime.now() - self.application_date).days, 0)
        return wait_time

    def __lt__(self, other):
        return self.score < other.score

In [4]:
class Doctor:
    def __init__(self, id, degree, experience, location, specialization, num_days=14):
        self.id = id
        self.degree = degree
        self.experience = experience
        self.location = location
        self.specialization = specialization
        self.availability = self.generate_availability(num_days)
        self.appointments = []
        self.emergency_quota = int(0.25 * len(self.availability))
        self.emergency_appointments = []

    def generate_availability(self, days):
        start_date = datetime.now().replace(hour=9, minute=0, second=0, microsecond=0)
        availability = []
        for day in range(days):
            current_date = start_date + timedelta(days=day)
            if current_date.weekday() < 5:  # Monday to Friday
                for hour in range(9, 17):  # 9 AM to 5 PM
                    availability.append(current_date.replace(hour=hour))
        return availability

    def view_schedule(self):
        schedule = []
        for slot in self.availability:
            status = "Available"
            for apt in self.appointments + self.emergency_appointments:
                if apt.datetime == slot:
                    status = f"Booked - Patient {apt.patient.id}"
                    break
            schedule.append((slot, status))
        return schedule

    def block_slot(self, slot):
        if slot in self.availability:
            self.availability.remove(slot)
            self.reschedule_affected_appointments(slot)
        else:
            print(f"Slot {slot} is not in your availability.")

    def take_leave(self, start_date, end_date):
        leave_slots = [slot for slot in self.availability if start_date <= slot <= end_date]
        for slot in leave_slots:
            if slot in self.availability:
                self.availability.remove(slot)
        self.reschedule_affected_appointments(leave_slots)

    def reschedule_affected_appointments(self, slots):
        if isinstance(slots, datetime):
            slots = [slots]
        affected_appointments = [apt for apt in self.appointments + self.emergency_appointments if apt.datetime in slots]
        for apt in affected_appointments:
            if apt in self.appointments:
                self.appointments.remove(apt)
            elif apt in self.emergency_appointments:
                self.emergency_appointments.remove(apt)
            appointment_system.reschedule_appointment(apt.patient)

    def __lt__(self, other):
        return self.score < other.score

In [5]:
class Appointment:
    def __init__(self, doctor, patient, datetime, is_emergency=False):
        self.doctor = doctor
        self.patient = patient
        self.datetime = datetime
        self.is_emergency = is_emergency

In [6]:
class MatchingSystem:
    def __init__(self):
        self.doctors = []
        self.embeddings = Embeddings()

    def add_doctor(self, doctor):
        self.doctors.append(doctor)
        self.update_embeddings()

    def update_embeddings(self):
        doctor_descriptions = [f"{doctor.degree} {doctor.specialization} {doctor.location}" for doctor in self.doctors]
        self.embeddings.index([(i, desc, None) for i, desc in enumerate(doctor_descriptions)])

    def find_best_match(self, patient):
        patient_description = f"{patient.required_doctor_type} {patient.location} {patient.text_description}"
        results = self.embeddings.search(patient_description, limit=len(self.doctors))

        best_matches = []
        for result in results:
            if isinstance(result, (list, tuple)) and len(result) >= 2:
                doctor_index, score = result[0], result[1]
            elif isinstance(result, dict):
                doctor_index, score = result.get('id'), result.get('score')
            else:
                continue  # Skip this result if it's in an unexpected format

            if doctor_index is not None and score is not None:
                doctor = self.doctors[doctor_index]
                if doctor.specialization == patient.required_doctor_type:
                    match_score = self.calculate_match_score(doctor, patient, score)
                    heapq.heappush(best_matches, (-match_score, doctor))

        return [heapq.heappop(best_matches)[1] for _ in range(min(3, len(best_matches)))]

    def calculate_match_score(self, doctor, patient, similarity):
        location_score = 1 if doctor.location == patient.location else 0
        experience_score = min(doctor.experience / 10, 1)
        urgency_score = patient.score / 7
        return similarity + location_score + experience_score + urgency_score

In [7]:
class AppointmentSystem:
    def __init__(self):
        self.waiting_list = []

    def schedule_appointment(self, doctor, patient, date):
        appointment = Appointment(doctor, patient, date, patient.is_emergency)
        if patient.is_emergency:
            if len(doctor.emergency_appointments) < doctor.emergency_quota:
                doctor.emergency_appointments.append(appointment)
            else:
                rescheduled = next((apt for apt in doctor.emergency_appointments if not apt.patient.is_emergency), None)
                if rescheduled:
                    doctor.emergency_appointments.remove(rescheduled)
                    self.reschedule_appointment(rescheduled.patient)
                doctor.emergency_appointments.append(appointment)
        else:
            if len(doctor.appointments) < len(doctor.availability) - doctor.emergency_quota:
                doctor.appointments.append(appointment)
            else:
                if len(doctor.emergency_appointments) < doctor.emergency_quota:
                    doctor.emergency_appointments.append(appointment)
                else:
                    return False

        patient.appointment = appointment
        patient.last_reschedule_date = None  # Reset last reschedule date when appointment is scheduled
        return True

    def reschedule_appointment(self, patient):
      if patient.appointment:
        old_doctor = patient.appointment.doctor
        # print(old_doctor)
        # print("+++++++++++++++++++++++++++++")
        if patient.appointment in old_doctor.appointments:
            old_doctor.appointments.remove(patient.appointment)
        elif patient.appointment in old_doctor.emergency_appointments:
            old_doctor.emergency_appointments.remove(patient.appointment)
        else:
          pass
            # print(f"Appointment for patient {patient.id} not found in doctor's schedule.")

        patient.appointment = None
        patient.last_reschedule_date = datetime.now()

    def add_to_waiting_list(self, patient):
        try:
            priority = -float(patient.score)
        except ValueError:
            print(f"Warning: Invalid score for patient {patient.id}. Using default priority 0.")
            priority = 0
        heapq.heappush(self.waiting_list, (priority, patient))

    def get_next_available_slot(self, doctor, is_emergency):
        all_appointments = doctor.appointments + doctor.emergency_appointments
        for slot in doctor.availability:
            if slot not in [apt.datetime for apt in all_appointments]:
                if is_emergency or len(all_appointments) < len(doctor.availability) - doctor.emergency_quota:
                    return slot
        return None

    def process_appointments(self, matching_system):
        while self.waiting_list:
            _, patient = heapq.heappop(self.waiting_list)
            best_doctors = matching_system.find_best_match(patient)

            scheduled = False
            for doctor in best_doctors:
                next_slot = self.get_next_available_slot(doctor, patient.is_emergency)
                if next_slot:
                    if self.schedule_appointment(doctor, patient, next_slot):
                        print(f"Scheduled appointment for patient {patient.id} with doctor {doctor.id} at {next_slot}")
                        print(f"Waiting time: {patient.waiting_time} days")
                        scheduled = True
                        break

            if not scheduled:
                print(f"No suitable slot found for patient {patient.id}. Adding back to waiting list.")
                print(f"Current waiting time: {patient.waiting_time} days")
                self.add_to_waiting_list(patient)

    def update_waiting_times(self):
        updated_list = []
        while self.waiting_list:
            score, patient = heapq.heappop(self.waiting_list)
            patient.score = min(patient.score + 0.1 * patient.waiting_time, 7)
            heapq.heappush(updated_list, (-patient.score, patient))
        self.waiting_list = updated_list

    def request_reschedule(self, patient, preferred_date=None):
        if patient.appointment:
            old_doctor = patient.appointment.doctor
            old_doctor.appointments.remove(patient.appointment)
            patient.appointment = None
            patient.last_reschedule_date = datetime.now()  # Set last reschedule date

            if preferred_date:
                if self.schedule_appointment(old_doctor, patient, preferred_date):
                    print(f"Rescheduled appointment for patient {patient.id} with doctor {old_doctor.id} at {preferred_date}")
                    print(f"New waiting time: {patient.waiting_time} days")
                    return

            self.add_to_waiting_list(patient)
            print(f"Patient {patient.id} added to waiting list for rescheduling")
            print(f"Current waiting time: {patient.waiting_time} days")
        else:
            print(f"Patient {patient.id} has no appointment to reschedule")

<h2>Sample Test Code 1</h2>

In [8]:
# Example usage:
matching_system = MatchingSystem()
appointment_system = AppointmentSystem()

In [9]:
# Create some doctors
doctor1 = Doctor(1, "MD", 10, "New York", "Cardiologist", num_days=14)
doctor2 = Doctor(2, "PhD", 15, "New York", "Cardiologist", num_days=5)
doctor3 = Doctor(3, "MD", 8, "Los Angeles", "Neurologist", num_days=18)
matching_system.add_doctor(doctor1)
matching_system.add_doctor(doctor2)
matching_system.add_doctor(doctor3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
# Create some patients
patient1 = Patient(1, "New York", "Stable", "video1.mp4", "Heart palpitations and shortness of breath", "Cardiologist", 5)
patient2 = Patient(2, "Los Angeles", "Critical", "video2.mp4", "Severe headache and confusion", "Neurologist", 7, is_emergency=True)
patient3 = Patient(3, "New York", "Stable", "video3.mp4", "Chest pain and dizziness", "Cardiologist", 6)

In [11]:
# Add patients to waiting list
appointment_system.add_to_waiting_list(patient1)
appointment_system.add_to_waiting_list(patient2)
appointment_system.add_to_waiting_list(patient3)

In [12]:
# Process appointments
appointment_system.process_appointments(matching_system)

# Update waiting times and reprocess
appointment_system.update_waiting_times()
appointment_system.process_appointments(matching_system)

# Simulate an emergency patient arrival
emergency_patient = Patient(4, "New York", "Critical", "video4.mp4", "Severe chest pain and shortness of breath", "Cardiologist", 7, is_emergency=True)
appointment_system.add_to_waiting_list(emergency_patient)
appointment_system.process_appointments(matching_system)

Scheduled appointment for patient 2 with doctor 3 at 2024-08-05 09:00:00
Waiting time: 0 days
Scheduled appointment for patient 3 with doctor 1 at 2024-08-05 09:00:00
Waiting time: 0 days
Scheduled appointment for patient 1 with doctor 1 at 2024-08-05 10:00:00
Waiting time: 0 days
Scheduled appointment for patient 4 with doctor 1 at 2024-08-05 11:00:00
Waiting time: 0 days


In [13]:
print(patient1.appointment)

In [14]:
# Generate the datetime object
preferred_date = datetime.now() + timedelta(days=3)

# Print the datetime in the desired format
# preferred_date = new_datetime.strftime("datetime.datetime(%Y, %m, %d, %H, %M)")

appointment_system.request_reschedule(patient1, preferred_date)
appointment_system.process_appointments(matching_system)

Rescheduled appointment for patient 1 with doctor 1 at 2024-08-08 15:35:21.296461
New waiting time: 3 days


<h2>For Mass patient Matching</h2>

In [15]:
def generate_dummy_doctors(num_doctors=50):
    locations = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose"]
    doctor_types = ["Cardiologist", "Neurologist", "Oncologist", "Pediatrician", "Dermatologist", "Orthopedist", "Gynecologist", "Urologist", "Psychiatrist", "Endocrinologist"]
    degrees = ["MD", "PhD", "DO", "MBBS"]

    doctors = []
    doctor_id = 1

    # Ensure at least one doctor for each specialization
    for doctor_type in doctor_types:
        doctor = Doctor(
            id=doctor_id,
            degree=random.choice(degrees),
            experience=random.randint(5, 30),
            location=random.choice(locations),
            specialization=doctor_type,
            # Can chage the number of days
            num_days=400
        )
        doctors.append(doctor)
        doctor_id += 1

    # Add additional random doctors to reach the desired number
    while len(doctors) < num_doctors:
        doctor = Doctor(
            id=doctor_id,
            degree=random.choice(degrees),
            experience=random.randint(5, 30),
            location=random.choice(locations),
            specialization=random.choice(doctor_types),
            # Can chage the number of days
            num_days=400
        )
        doctors.append(doctor)
        doctor_id += 1

    return doctors

In [16]:
# Generate dummy doctors
dummy_doctors = generate_dummy_doctors(num_doctors=12)

# Assuming the matching_system and its add_doctor method are already defined
for doctor in dummy_doctors:
    matching_system.add_doctor(doctor)

# Print a few examples to verify
for doctor in dummy_doctors:
    print(f"Doctor {doctor.id}: {doctor.degree}, {doctor.experience} years, {doctor.location}, {doctor.specialization}")


Doctor 1: DO, 30 years, Chicago, Cardiologist
Doctor 2: DO, 11 years, Philadelphia, Neurologist
Doctor 3: PhD, 13 years, Philadelphia, Oncologist
Doctor 4: PhD, 18 years, Houston, Pediatrician
Doctor 5: MBBS, 19 years, Houston, Dermatologist
Doctor 6: MBBS, 14 years, New York, Orthopedist
Doctor 7: DO, 19 years, New York, Gynecologist
Doctor 8: DO, 7 years, Los Angeles, Urologist
Doctor 9: DO, 24 years, Houston, Psychiatrist
Doctor 10: MD, 8 years, Chicago, Endocrinologist
Doctor 11: PhD, 14 years, Los Angeles, Orthopedist
Doctor 12: MD, 13 years, Chicago, Pediatrician


In [17]:
def generate_dummy_patients(num_patients=100):
    locations = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose"]
    health_statuses = ["Stable", "Moderate", "Serious", "Critical"]
    doctor_types = ["Cardiologist", "Neurologist", "Oncologist", "Pediatrician", "Dermatologist", "Orthopedist", "Gynecologist", "Urologist", "Psychiatrist", "Endocrinologist"]

    symptoms = {
        "Cardiologist": ["Heart palpitations", "Chest pain", "Shortness of breath", "Dizziness", "Fainting", "Swollen legs"],
        "Neurologist": ["Severe headache", "Confusion", "Numbness", "Tingling sensations", "Balance problems", "Seizures"],
        "Oncologist": ["Unexplained weight loss", "Fatigue", "Fever", "Pain", "Skin changes", "Persistent cough"],
        "Pediatrician": ["Fever", "Cough", "Rash", "Ear pain", "Sore throat", "Abdominal pain"],
        "Dermatologist": ["Skin rash", "Acne", "Skin discoloration", "Itching", "Mole changes", "Hair loss"],
        "Orthopedist": ["Joint pain", "Back pain", "Fractures", "Sprains", "Arthritis", "Osteoporosis"],
        "Gynecologist": ["Abnormal bleeding", "Pelvic pain", "Breast lumps", "Vaginal discharge", "Menstrual disorders", "Pregnancy concerns"],
        "Urologist": ["Urinary incontinence", "Kidney stones", "Erectile dysfunction", "Prostate issues", "Bladder infections", "Blood in urine"],
        "Psychiatrist": ["Depression", "Anxiety", "Panic attacks", "Mood swings", "Insomnia", "Substance abuse"],
        "Endocrinologist": ["Diabetes symptoms", "Thyroid problems", "Hormonal imbalances", "Weight management issues", "Osteoporosis", "Adrenal disorders"]
    }

    patients = []
    for i in range(1, num_patients + 1):
        location = random.choice(locations)
        health_status = random.choice(health_statuses)
        doctor_type = random.choice(doctor_types)
        symptom1, symptom2 = random.sample(symptoms[doctor_type], 2)
        text_description = f"{symptom1} and {symptom2}"
        score = float(random.randint(1, 7))
        is_emergency = health_status == "Critical" or random.random() < 0.1  # 10% chance of being emergency even if not critical

        patient = Patient(
            id=i,
            location=location,
            health_status=health_status,
            video_description=f"video{i}.mp4",
            text_description=text_description,
            required_doctor_type=doctor_type,
            score=score,
            is_emergency=is_emergency
        )
        patients.append(patient)

    return patients

In [18]:
# Generate 100 dummy patients
dummy_patients = generate_dummy_patients(1000)

# Print a few examples to verify
for patient in dummy_patients[:10]:
    print(f"Patient {patient.id}: {patient.location}, {patient.health_status}, {patient.required_doctor_type}, "
          f"Emergency: {patient.is_emergency}, Score: {patient.score}, Symptoms: {patient.text_description}")

print("Length of the dummy patients: ", len(dummy_patients))

Patient 1: San Antonio, Serious, Orthopedist, Emergency: False, Score: 7.0, Symptoms: Back pain and Osteoporosis
Patient 2: Los Angeles, Critical, Pediatrician, Emergency: True, Score: 2.0, Symptoms: Sore throat and Rash
Patient 3: Philadelphia, Stable, Gynecologist, Emergency: False, Score: 7.0, Symptoms: Vaginal discharge and Breast lumps
Patient 4: San Diego, Critical, Psychiatrist, Emergency: True, Score: 7.0, Symptoms: Insomnia and Mood swings
Patient 5: Houston, Critical, Psychiatrist, Emergency: True, Score: 6.0, Symptoms: Anxiety and Insomnia
Patient 6: San Antonio, Critical, Endocrinologist, Emergency: True, Score: 4.0, Symptoms: Weight management issues and Hormonal imbalances
Patient 7: New York, Stable, Urologist, Emergency: False, Score: 6.0, Symptoms: Erectile dysfunction and Blood in urine
Patient 8: San Diego, Stable, Urologist, Emergency: False, Score: 3.0, Symptoms: Blood in urine and Erectile dysfunction
Patient 9: San Diego, Moderate, Pediatrician, Emergency: False,

In [19]:
# Add patients to waiting list
for patient in dummy_patients:
  appointment_system.add_to_waiting_list(patient)

In [20]:
# Process appointments
appointment_system.process_appointments(matching_system)

Scheduled appointment for patient 1 with doctor 11 at 2024-08-05 09:00:00
Waiting time: 0 days
Scheduled appointment for patient 3 with doctor 7 at 2024-08-05 09:00:00
Waiting time: 0 days
Scheduled appointment for patient 121 with doctor 7 at 2024-08-05 10:00:00
Waiting time: 0 days
Scheduled appointment for patient 240 with doctor 10 at 2024-08-05 09:00:00
Waiting time: 0 days
Scheduled appointment for patient 253 with doctor 9 at 2024-08-05 09:00:00
Waiting time: 0 days
Scheduled appointment for patient 506 with doctor 1 at 2024-08-05 09:00:00
Waiting time: 0 days
Scheduled appointment for patient 507 with doctor 8 at 2024-08-05 09:00:00
Waiting time: 0 days
Scheduled appointment for patient 502 with doctor 10 at 2024-08-05 10:00:00
Waiting time: 0 days
Scheduled appointment for patient 993 with doctor 7 at 2024-08-05 11:00:00
Waiting time: 0 days
Scheduled appointment for patient 485 with doctor 10 at 2024-08-05 11:00:00
Waiting time: 0 days
Scheduled appointment for patient 978 wi

In [21]:
# # Update waiting times and reprocess
# appointment_system.update_waiting_times()
# appointment_system.process_appointments(matching_system)

<h2>Added the new functionality of the doctor slot blocking, doctor leave and scheduling viewing</h2>

In [22]:
doctor1.availability[0]

datetime.datetime(2024, 8, 5, 9, 0)

In [23]:
# doctor1 = dummy_doctors[300]

# Example of a doctor blocking a slot
print("\nBlocking a slot for Doctor 1:")
slot_to_block = doctor1.availability[0] # Instead of the 0 put here the id of the slot that is to be blocked, and the appoitment is to be made cancelled
doctor1.block_slot(slot_to_block)
print(f"Blocked slot: {slot_to_block}")


Blocking a slot for Doctor 1:
Blocked slot: 2024-08-05 09:00:00


In [24]:
doctor1.availability[3]

datetime.datetime(2024, 8, 5, 13, 0)

In [25]:
from datetime import datetime, timedelta

# Set the start date for the leave (for example, starting tomorrow)
start_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1)

# Set the end date for the leave (for example, 5 days from the start date)
end_date = start_date + timedelta(days=5)

# Call the take_leave function
doctor1.take_leave(start_date, end_date)

# Print confirmation
print(f"Doctor 1 is on leave from {start_date.date()} to {end_date.date()} and all apointments cancelled")

Doctor 1 is on leave from 2024-08-06 to 2024-08-11 and all apointments cancelled


In [26]:
# Process appointments again after changes
print("\nProcessing appointments after schedule changes:")
appointment_system.process_appointments(matching_system)

# Update waiting times and scores
appointment_system.update_waiting_times()

# Process appointments one more time
print("\nFinal appointment processing:")
appointment_system.process_appointments(matching_system)


Processing appointments after schedule changes:

Final appointment processing:


In [27]:
dummy_patients[989].appointment.datetime

datetime.datetime(2024, 8, 14, 11, 0)

In [28]:
dummy_patients[989].id

990

In [29]:
# Example of requesting a reschedule

# Generate the datetime object
preferred_date = datetime.now() + timedelta(days=3)

patient_to_reschedule = dummy_patients[989]
print("\nRequesting reschedule:")
appointment_system.request_reschedule(patient_to_reschedule, preferred_date=preferred_date)

# Process appointments after rescheduling request
print("\nProcessing appointments after rescheduling request:")
appointment_system.process_appointments(matching_system)


Requesting reschedule:
Rescheduled appointment for patient 990 with doctor 3 at 2024-08-08 15:36:01.187447
New waiting time: 3 days

Processing appointments after rescheduling request:


In [30]:
dummy_patients[989].appointment.datetime

datetime.datetime(2024, 8, 8, 15, 36, 1, 187447)

In [31]:
# Checing Availability of the doctor
doctor1.view_schedule()

[(datetime.datetime(2024, 8, 5, 10, 0), 'Booked - Patient 475'),
 (datetime.datetime(2024, 8, 5, 11, 0), 'Booked - Patient 4'),
 (datetime.datetime(2024, 8, 5, 12, 0), 'Booked - Patient 817'),
 (datetime.datetime(2024, 8, 5, 13, 0), 'Booked - Patient 11'),
 (datetime.datetime(2024, 8, 5, 14, 0), 'Booked - Patient 735'),
 (datetime.datetime(2024, 8, 5, 15, 0), 'Booked - Patient 170'),
 (datetime.datetime(2024, 8, 5, 16, 0), 'Booked - Patient 331'),
 (datetime.datetime(2024, 8, 12, 9, 0), 'Booked - Patient 188'),
 (datetime.datetime(2024, 8, 12, 10, 0), 'Booked - Patient 613'),
 (datetime.datetime(2024, 8, 12, 11, 0), 'Booked - Patient 342'),
 (datetime.datetime(2024, 8, 12, 12, 0), 'Booked - Patient 607'),
 (datetime.datetime(2024, 8, 12, 13, 0), 'Booked - Patient 537'),
 (datetime.datetime(2024, 8, 12, 14, 0), 'Booked - Patient 943'),
 (datetime.datetime(2024, 8, 12, 15, 0), 'Booked - Patient 929'),
 (datetime.datetime(2024, 8, 12, 16, 0), 'Booked - Patient 395'),
 (datetime.datetime(2

<h2>For Getting out the details of the particular patient (For checking / Debuging)</h2>

In [32]:
# doctor1.id

In [33]:
# patient1.appointment.doctor.id

In [34]:
# patient1.appointment.doctor.experience

In [35]:
# patient1.appointment.patient.id

In [36]:
# doctor1.appointments[0].patient.id

In [37]:
# dummy_patients[191].appointment.doctor.id